In [ ]:
class desafioBiodiversidade():

    
    #1
    #Para cada coluna identique a quantidade de linhas com dados faltantes (em alguns casos, 
    #o dado faltante é uma string vazia, em outros casos é uma string contendo algum valor do 
    #tipo: "sem informação"). 
    #Faça um método que retorna a média de dados faltantes por coluna
    def retornaMediaDadosFaltantesPorColuna():
        
    
    #2
    #Para cada item identifique até qual nível taxônomico a ocorrência foi identificada.
    #Seis campos definem a espécie: Filo;Classe;Ordem;Familia;Genero;Especie do mais genérico 
    #(Filo) para o mais específico (Especie), é comum que em alguns casos, o preenchimento pare em 
    #família ou genêro por exemplo.
   
    
    def retornaNivelTaxonomicoCadaOcorrencia():
    
    
    #3
    #Monte filtros de ocorrências por estados, nome de espécie (nome exato ou parte do nome) e 
    #categoria de ameaça, e outros filtros que julgar relevante.
    def retonaSeOcorrenciaExiste():
        
        
    #4
    #Crie uma funcionalidade para avaliar se a informação de longitude e latitude 
    #corresponde a informação presente na localização, para isso você pode utilizar uma 
    #biblioteca de consulta reversa de lat/log como por exemplo o 
    #https://opencagedata.com/tutorials/geocode-in-python
    def metodoPlotMap():
        

biodiversidade = desafioBiodiversidade()

In [1]:
import csv

class Especie():
    def __init__(self, dados_tax): 
        self.lista_tax = dados_tax
        #print(dados_tax)

    def retornaNivelTaxonomicoCadaOcorrencia(self):
        lista = []
        for i in self.lista_tax:
            lista.append(i.strip("\n").split(";"))

        filo = []
        for i in range(len(lista[0])):
            
            if lista[0][i] == 'Especie':
              
                for k in range(1,len(lista)):
                    
                    if lista[k][i] != 'Sem Informações':
                        filo.append('{} - Especie'.format(k))
                    
                    elif lista[k][i-1] != 'Sem Informações':
                        filo.append('{} - Genero'.format(k))
                        
                    elif lista[k][i-2] != 'Sem Informações':
                        filo.append('{} - Familia'.format(k))
                        
                    elif lista[k][i-3] != 'Sem Informações':
                        filo.append('{} - Ordem'.format(k))
                    
                    elif lista[k][i-4] != 'Sem Informações':
                        filo.append('{} - Classe'.format(k))
                        
                    elif lista[k][i-5] != 'Sem Informações':
                        filo.append('{} - Filo'.format(k))
                        
                    else:
                        filo.append('{} - Reino'.format(k))
     
        return filo  

file_name = "portalbio_export_17-10-2019-13-06-22.csv"

identif_taxonomico = open(file_name , 'r')

lista_especie = identif_taxonomico.readlines() #le as linhas


tax = Especie(lista_especie)

#chamando os metodos

tax.retornaNivelTaxonomicoCadaOcorrencia()


['1 - Especie',
 '2 - Especie',
 '3 - Especie',
 '4 - Especie',
 '5 - Genero',
 '6 - Especie',
 '7 - Especie',
 '8 - Especie',
 '9 - Especie',
 '10 - Especie',
 '11 - Especie',
 '12 - Especie',
 '13 - Especie',
 '14 - Especie',
 '15 - Especie',
 '16 - Especie',
 '17 - Especie',
 '18 - Especie',
 '19 - Especie',
 '20 - Especie',
 '21 - Especie',
 '22 - Especie',
 '23 - Especie',
 '24 - Especie',
 '25 - Especie',
 '26 - Especie',
 '27 - Especie',
 '28 - Especie',
 '29 - Ordem',
 '30 - Familia',
 '31 - Genero',
 '32 - Especie',
 '33 - Especie',
 '34 - Especie',
 '35 - Especie',
 '36 - Especie',
 '37 - Especie',
 '38 - Especie',
 '39 - Especie',
 '40 - Especie',
 '41 - Especie',
 '42 - Especie',
 '43 - Especie',
 '44 - Especie',
 '45 - Especie',
 '46 - Especie',
 '47 - Especie',
 '48 - Especie',
 '49 - Especie',
 '50 - Especie',
 '51 - Especie',
 '52 - Especie',
 '53 - Especie',
 '54 - Especie',
 '55 - Especie',
 '56 - Especie',
 '57 - Especie',
 '58 - Especie',
 '59 - Especie',
 '60 - Esp